# 필로그(Feel-Log) — 데이터 EDA & 전처리 노트북

이 노트북은 **완전 초보자를 위한 셀 단위 가이드**입니다. 각 셀을 **위에서부터 차례대로** 실행하세요 (`Shift+Enter`).  
윈도우 경로가 이미 반영되어 있으며, 필요하면 첫 번째 셀에서 `BASE_DIR`만 바꾸면 됩니다.


## 02 — 감정별 이미지 폴더 EDA

In [ ]:
# 버전 충돌 방지를 위해 호환 버전으로 설치 (한 번만 실행하면 됩니다)
!pip install -U "pip<25.3"
!pip install -U "numpy==1.26.4" \
               "pandas==2.2.2" \
               "matplotlib==3.8.4" \
               "pillow==10.3.0" \
               "scikit-learn==1.4.2" \
               "opencv-python==4.10.0.84"


In [ ]:
import os, json, glob
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

# 그래프 한글 폰트 이슈가 있으면 주석 해제하고 시스템에 맞는 폰트를 지정하세요.
# import matplotlib
# matplotlib.rc('font', family='Malgun Gothic')  # 윈도우 '맑은 고딕'
# plt.rcParams['axes.unicode_minus'] = False


In [ ]:
# Windows 한글 폰트 설정 (맑은 고딕)
from matplotlib import rcParams, font_manager
try:
    font_manager.fontManager.addfont(r"C:\Windows\Fonts\malgun.ttf")
    rcParams["font.family"] = "Malgun Gothic"
    rcParams["axes.unicode_minus"] = False
    print("Matplotlib font set to Malgun Gothic.")
except Exception as e:
    print("폰트 설정 실패:", e)


In [ ]:
\
# ★★★ 여기만 확인/수정하세요 ★★★
# 윈도우의 실제 데이터 폴더
BASE_DIR = Path(r"C:\Users\USER\Downloads\korean_emotion_complex_vision_1_percent")

# 서브 폴더들 (변경하지 않아도 됨)
LABELS_DIR = BASE_DIR / "labels"                # JSON들이 들어있는 폴더
EMOTION_DIR = BASE_DIR                           # 감정별 이미지 폴더들이 바로 아래에 있음 (중립/슬픔/상처/불안/분노/당황/기쁨)
PROCESSED_DIR = BASE_DIR / "processed"           # 전처리 결과 저장 폴더(자동 생성)
(PROCESSED_DIR / "from_json").mkdir(parents=True, exist_ok=True)
(PROCESSED_DIR / "from_folders").mkdir(parents=True, exist_ok=True)

BASE_DIR, LABELS_DIR, EMOTION_DIR, PROCESSED_DIR


### 1) 감정 폴더 목록 스캔

In [ ]:
emotions = []
for p in EMOTION_DIR.iterdir():
    if p.is_dir() and p.name not in ['labels', 'processed']:
        emotions.append(p.name)
emotions


### 2) 이미지 메타데이터 테이블 만들기

In [ ]:
records = []
for em in emotions:
    em_dir = EMOTION_DIR / em
    for ext in ('*.jpg','*.jpeg','*.png','*.bmp','*.webp'):
        for img_path in em_dir.glob(ext):
            try:
                with Image.open(img_path) as im:
                    w,h = im.size
                    mode = im.mode
                records.append({'emotion':em, 'path':str(img_path), 'width':w, 'height':h, 'mode':mode})
            except Exception as e:
                records.append({'emotion':em, 'path':str(img_path), 'width':None, 'height':None, 'mode':None, 'error':str(e)})
df_img = pd.DataFrame(records)
df_img.head()


### 3) 클래스별 개수와 해상도 분포

In [ ]:
display(df_img.groupby('emotion')['path'].count().sort_values(ascending=False))
plt.figure(figsize=(6,4))
df_img['width'].dropna().hist(bins=30)
plt.title('이미지 가로 길이 분포')
plt.xlabel('width')
plt.ylabel('count')
plt.xticks(rotation=0)
plt.show()


### 4) 샘플 이미지 썸네일 저장 (각 클래스 최대 3개)

In [ ]:
sample_out = BASE_DIR / "samples"
sample_out.mkdir(exist_ok=True)
for em, g in df_img.groupby('emotion'):
    sample = g['path'].dropna().sample(min(3,len(g)), random_state=42) if len(g)>0 else []
    for i, p in enumerate(sample):
        with Image.open(p) as im:
            im.thumbnail((256,256))
            im.save(sample_out / f"{em}_{i}.jpg")
list(sample_out.iterdir())[:10]


### 5) 결과 CSV 저장 (선택)

In [ ]:
out_csv = BASE_DIR / "images_eda_table.csv"
df_img.to_csv(out_csv, index=False)
out_csv
